# CSC/XMM validation catalogs

In [21]:
import sys
sys.path.append('../')

In [22]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np

import matplotlib.pyplot as plt
# plt.style.use('seaborn')

import seaborn as sns

from scripts.utils import set_mpl, data_path
from scripts.cross_match_scripts import cross_match_data_frames,  add_separation_columns
set_mpl()

%matplotlib inline
from astropy.table import Table

matplotlib settings set


***

In [23]:
ero_df = pd.read_pickle(data_path+'ERO_lhpv_03_23_sd01_a15_g14.pkl') 
ero_df.sample(5)

,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH
2385,SRGe J103158.7+585504,157.994460,58.917783,150.343903,50.131188,5.860175,28.274986,1.296847e-14,2.787584e-15,35.579102,7.647758,2506.418945,0.0,0.0,0.0,3382,2695,157.996468,58.916494,2.479020,41.212132,0.000013,6.268413e+08,6.270701e+08,0.0,0.0,0.014195,0.003051,12.144133,157.994460,58.917783,1047283386643601792,7.521387,1,1,10.046179,NaN,0,BD+59 1313,0.000027,Star,7.367067,1,0,0,0,9.188,9.196000,9.139,48.6,55.4,35.0,1,2,0,1237653616933535775,0,7.330820,59197545632,2.479020,2.095052,3.162674,5.128157,132.731210,45.138122,9.979746e-15,7.875664e+19
3872,SRGe J104239.8+573310,160.665950,57.552910,150.651058,52.081177,5.717220,37.199787,8.332741e-15,1.635746e-15,39.290443,7.712849,4307.710938,0.0,0.0,0.0,3127,2489,160.667959,57.551620,2.411232,142.322861,0.000010,6.268413e+08,6.270701e+08,0.0,0.0,0.009121,0.001790,16.733130,160.665950,57.552910,-1,-1.000000,0,-1,NaN,-1.0,-1,NaN,NaN,NaN,-1.000000,0,0,1,0,17.034,16.889000,12.416,10.6,3.3,0.8,1,2,0,1237658303279071756,0,7.598983,64367246269,2.411232,2.043945,3.085523,5.003059,135.434049,44.716131,6.412369e-15,5.852172e+19
20,SRGe J103037.3+555849,157.655561,55.980279,154.292674,51.702219,2.064971,6574.125977,3.553128e-13,8.488733e-15,1951.615601,46.625797,5018.002930,0.0,0.0,0.0,19,12,157.657570,55.978989,0.347972,79.215965,0.000004,6.268413e+08,6.270701e+08,0.0,0.0,0.388923,0.009292,21.759691,157.655561,55.980279,853819947756949120,6.990223,1,1,4.673892,NaN,1,* 36 UMa,0.000028,PM*,5.923889,1,1,0,0,3.562,3.276000,3.577,2.4,3.4,69.7,1,0,-1,0,0,NaN,70570690035,0.347972,0.738241,1.114443,1.807027,134.594706,42.519703,2.734271e-13,6.019163e+19
1660,SRGe J105240.7+554239,163.169730,55.710948,151.515053,54.321746,7.301241,18.246275,1.716180e-14,4.594207e-15,22.709171,6.079235,1208.887451,0.0,0.0,0.0,5417,4431,163.171738,55.709659,3.153486,131.581604,0.000016,6.268413e+08,6.270701e+08,0.0,0.0,0.018785,0.005029,7.226747,163.169730,55.710948,-1,-1.000000,0,-1,NaN,-1.0,-1,NaN,NaN,NaN,-1.000000,0,0,0,-1,NaN,NaN,NaN,NaN,NaN,NaN,0,0,-1,0,0,NaN,71658344194,3.153486,2.610243,3.940403,6.389215,138.427055,43.879600,1.320668e-14,9.660095e+19
5366,SRGe J104230.3+553519,160.626334,55.588705,153.210344,53.286292,7.997332,16.477905,5.947508e-15,1.543391e-15,28.497999,7.395293,4377.506836,0.0,0.0,0.0,6709,5611,160.628343,55.587415,3.475267,114.323486,0.000004,6.268413e+08,6.270701e+08,0.0,0.0,0.006510,0.001689,18.442711,160.626334,55.588705,-1,-1.000000,0,-1,NaN,-1.0,-1,NaN,NaN,NaN,-1.000000,0,0,0,0,16.354,16.264999,12.272,18.1,6.3,3.3,1,1,0,1237657222022955305,0,8.199435,72155126074,3.475267,2.859101,4.316076,6.998354,136.827152,43.027845,4.576839e-15,5.978287e+19


In [24]:
# desi_lh.gz_pkl number of sources and area
false_dens_arcmin = 2418574 / (41.729 * 3600)
print(false_dens_arcmin)
false_dens_arcsec = false_dens_arcmin / 3600

print(f'{ false_dens_arcsec:.3f}')

FALSE_PROB = .03



def fsource_r(rho: float, f_thresh: float) -> float:
    """
    The function calculates the radius which corresponds to the probability `f_thresh`
    to FIND one or more false sources. 

    Args:
        rho (float): the density of the false sources (in arcmin^{-2}).
        f_thresh (float): probability to FIND one or more false sources.

    Returns:
        float: radius (in arcsec).
    """

    rho = rho / 3600 # arcmin^{-2} -> arcsec^{-2}
    pf_r = np.sqrt(-1 / (rho * np.pi) * np.log(1 - f_thresh))
    
    return pf_r




r_false = fsource_r(false_dens_arcmin, FALSE_PROB)

print(f'Радиус для {FALSE_PROB:.0%} вероятности найти ложный: {r_false:.2f}"')

desi = pd.read_pickle(data_path+'desi_lh.gz_pkl', compression='gzip')
desi.sample(5)

16.099741453452303
0.004
Радиус для 3% вероятности найти ложный: 1.47"


,release,objid,brickid,ra,dec,flux_g,flux_r,flux_z,flux_w1,flux_w2,flux_w3,flux_w4,flux_ivar_g,flux_ivar_r,flux_ivar_z,flux_ivar_w1,flux_ivar_w2,flux_ivar_w3,flux_ivar_w4,dered_mag_g,dered_mag_r,dered_mag_z,dered_mag_w1,dered_mag_w2,dered_mag_w3,dered_mag_w4,snr_g,snr_r,snr_z,snr_w1,snr_w2,snr_w3,snr_w4,type,parallax,parallax_ivar,pmra,pmra_ivar,pmdec,pmdec_ivar,ref_cat,ref_id,mjd_max,mjd_min,iso_max,iso_min,desi_id
36410,9011,2624,622390,162.041499,61.735105,0.345750,0.609436,1.059051,1.784355,-0.375681,12.818885,553.49880,361.75476,119.140970,34.876360,4.066398,1.095441,0.001309,0.000016,23.612911,23.010610,22.422567,21.868996,NaN,19.730074,15.642095,6.576114,6.652101,6.254352,3.598208,-0.393200,0.463817,2.243988,REX,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,58245.189944,57414.418241,2018-05-07 04:33:31.130,2016-01-27 10:02:16.000,9011_622390_2624
803381,9011,2453,611505,168.069522,58.097346,0.404764,0.749786,1.196152,0.774897,0.121960,29.737740,1152.77050,325.54710,117.031920,96.811325,4.069666,1.118613,0.001029,0.000013,23.451870,22.792364,22.294184,22.775166,24.783396,18.816504,14.845557,7.303133,8.111282,11.769269,1.563233,0.128991,0.953924,4.088951,REX,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,57895.181835,57784.463687,2017-05-22 04:21:50.501,2017-01-31 11:07:42.548,9011_611505_2453
1564586,9011,3389,605997,163.825926,56.219582,0.965411,0.946520,0.448091,3.973360,6.304521,-225.451190,-1365.50900,111.90081,29.836609,8.144178,3.394806,0.885592,0.001250,0.000018,22.504680,22.537083,23.358948,21.000185,20.499691,NaN,NaN,10.212423,5.170168,1.278761,7.320915,5.932927,-7.970050,-5.798820,REX,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,57916.200787,57809.407133,2017-06-12 04:49:08.000,2017-02-25 09:46:16.291,9011_605997_3389
1802596,9011,3882,603562,163.697903,55.496528,0.301122,0.618757,1.020022,1.237215,1.347783,-30.118927,-316.94443,962.93243,229.058910,61.781440,4.784450,1.149715,0.001591,0.000020,23.774136,23.001660,22.467546,22.267225,22.174930,NaN,NaN,9.344163,9.364694,8.017490,2.706208,1.445158,-1.201258,-1.406309,REX,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,58198.397609,57510.180225,2018-03-21 09:32:33.421,2016-05-02 04:19:31.450,9011_603562_3882
1172493,9011,3229,612245,156.820231,58.292153,0.124916,0.118567,0.491542,1.121891,0.335485,-4.829279,-340.54962,757.29700,251.604370,120.735630,4.686000,1.153292,0.001353,0.000017,24.711033,24.783154,23.253231,22.372410,23.684150,NaN,NaN,3.437580,1.880707,5.401053,2.428576,0.360282,-0.177605,-1.384658,PSF,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,58194.391325,57820.343785,2018-03-17 09:23:30.460,2017-03-08 08:15:03.000,9011_612245_3229


# CSC

In [25]:
csc_df = pd.read_pickle(data_path+'csc_all_lh.pkl')
csc_df.sample(5)

,name,ra,dec,err_ellipse_r0,err_ellipse_r1,err_ellipse_ang,significance,likelihood,likelihood_class,conf_flag,dither_warning_flag,extent_flag,pileup_flag,sat_src_flag,streak_src_flag,var_flag,flux_aper_s,flux_aper_lolim_s,flux_aper_hilim_s,flux_aper_m,flux_aper_lolim_m,flux_aper_hilim_m,radec_err,r_98,flux_05_2,flux_05_2_err,secure
127371,2CXO J105658.7+572158,164.244723,57.366364,2.278719,2.278721,0.000000,3.294118,37.158633,TRUE,False,False,False,False,False,False,False,1.333308e-14,7.936360e-15,1.872981e-14,1.753881e-15,3.653918e-16,3.142369e-15,1.316557,3.682606,1.508697e-14,7.880677e-15,True
126299,2CXO J105103.9+573054,162.766454,57.515235,1.063484,0.985573,109.602718,3.117647,158.928909,TRUE,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,0.592361,1.656921,NaN,NaN,True
126849,2CXO J105332.6+574316,163.385898,57.721310,3.276820,3.276820,0.000000,2.358974,20.454308,TRUE,False,False,False,False,False,False,False,3.228069e-15,1.268170e-15,5.187968e-15,2.875503e-15,1.189863e-15,4.561142e-15,1.893220,5.295619,6.103572e-15,3.655841e-15,True
114442,2CXO J103231.8+542404,158.132559,54.401132,0.756759,0.751906,26.251313,5.411765,24.029948,TRUE,False,False,True,False,False,False,False,5.724541e-15,4.483075e-15,6.966008e-15,2.379365e-15,1.542181e-15,3.172487e-15,0.435826,1.219072,8.103907e-15,2.100571e-15,False
115079,2CXO J103523.2+572220,158.846933,57.372309,4.300202,2.989757,1.555930,4.971429,92.750241,TRUE,False,False,False,False,False,False,True,5.242467e-15,4.062912e-15,6.422022e-15,5.471154e-15,4.047155e-15,6.895152e-15,2.139683,5.985013,1.071362e-14,2.615004e-15,True


## CSC x ERO cross-match

In [26]:
ero_csc_orig = cross_match_data_frames(ero_df, csc_df,
 'RA_fin', 'DEC_fin', 'ra', 'dec', match_radius = 30, df_prefix = 'csc')
 
ero_csc = ero_csc_orig.query("csc_n_near==1 & csc_n_matches==1")

ero_csc = ero_csc.query("csc_secure==True")
print('SECURITY FILTER APPLIED (CSC)')
print('cross-matches: only one csc within 30 arcsec and it is unique for the eROSITA source', ero_csc.shape[0])
assert len(ero_csc) == ero_csc.csc_name.nunique()
ero_csc.sample(5)

cross-match radius 30 arcsec
total matches: 777 out of 6885 x 2329
	 total unique pairs: 722
	 total non-unique pairs (duplicates in df2): 55
SECURITY FILTER APPLIED (CSC)
cross-matches: only one csc within 30 arcsec and it is unique for the eROSITA source 461


,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH,csc_name,csc_ra,csc_dec,csc_err_ellipse_r0,csc_err_ellipse_r1,csc_err_ellipse_ang,csc_significance,csc_likelihood,csc_likelihood_class,csc_conf_flag,csc_dither_warning_flag,csc_extent_flag,csc_pileup_flag,csc_sat_src_flag,csc_streak_src_flag,csc_var_flag,csc_flux_aper_s,csc_flux_aper_lolim_s,csc_flux_aper_hilim_s,csc_flux_aper_m,csc_flux_aper_lolim_m,csc_flux_aper_hilim_m,csc_radec_err,csc_r_98,csc_flux_05_2,csc_flux_05_2_err,csc_secure,csc_sep,csc_n_near,csc_n_matches
54,SRGe J104511.1+591626,161.296197,59.273939,148.202238,51.208144,3.038039,417.953094,4.841820e-14,3.569348e-15,227.581390,16.777105,4294.134766,0.0,0.0,0.0,404,341,161.298205,59.272650,1.056256,153.004074,0.000007,6.268413e+08,6.270701e+08,0.0,0.0,0.052998,0.003907,17.665037,161.296197,59.273939,855554083749377536,4.952248,1,0,19.455166,-1.0,0,[VV2006] J104511.2+591625,1.794480,QSO,4.951833,1,1,0,1,15.930000,14.703,11.867,22.1,21.3,4.7,1,2,1,1237655368746008682,631686511091279872,5.017947,57880912967,1.056256,1.086120,1.639598,2.658546,134.512266,46.345239,3.725969e-14,8.715714e+19,2CXO J104511.1+591625,161.296561,59.273768,0.712503,0.711654,70.202361,14.257766,3956.822546,TRUE,False,False,False,False,False,False,False,NaN,NaN,NaN,2.074348e-14,1.922112e-14,2.217069e-14,0.411412,1.150780,NaN,NaN,True,0.911220,1,1
707,SRGe J104918.6+585354,162.327671,58.898419,148.072605,51.852590,8.375615,13.915094,5.305120e-15,1.474734e-15,24.746695,6.879164,4261.568848,0.0,0.0,0.0,5429,4442,162.329680,58.897130,3.649390,57.704067,0.000006,6.268413e+08,6.270701e+08,0.0,0.0,0.005807,0.001614,21.352106,162.327671,58.898419,855305594121744000,7.283743,2,0,21.456654,-1.0,0,CXOSW J104918.9+585357,0.210000,Galaxy,7.626858,1,1,1,0,14.519000,14.091,10.799,35.9,27.3,11.0,1,2,0,1237655109446664427,0,7.207218,59269145415,3.649390,2.994339,4.520231,7.329384,135.452246,46.320982,4.082497e-15,8.326107e+19,2CXO J104918.8+585357,162.328458,58.899207,1.299884,1.127935,137.424103,6.178020,116.996233,TRUE,False,False,False,False,False,False,False,1.990220e-15,1.435569e-15,2.544872e-15,1.689500e-15,1.240727e-15,2.111875e-15,0.703107,1.966695,3.679720e-15,9.975344e-16,True,3.191994,1,1
434,SRGe J104852.0+584520,162.216797,58.755443,148.305700,51.907647,7.091814,35.781631,9.113958e-15,1.790819e-15,42.657375,8.381828,4275.969238,0.0,0.0,0.0,2742,2184,162.218805,58.754153,3.056259,235.055222,0.000012,6.268413e+08,6.270701e+08,0.0,0.0,0.009976,0.001960,16.566618,162.216797,58.755443,855300684975233664,5.543533,1,1,10.042479,NaN,0,HD 237932,NaN,Star,5.547432,1,1,0,0,9.174000,9.189,9.145,47.7,55.3,37.4,1,2,0,1237658304353337375,0,5.388772,59801926954,3.056259,2.535372,3.827377,6.205947,135.496089,46.169930,7.013546e-15,8.301948e+19,2CXO J104852.1+584520,162.217458,58.755808,2.603051,1.801656,144.048742,3.294118,32.708128,TRUE,False,False,False,False,False,False,False,2.015942e-15,1.403960e-15,2.591925e-15,2.142556e-16,4.657730e-17,3.819339e-16,1.293324,3.617620,2.230198e-15,8.732190e-16,True,1.804121,1,1
286,SRGe J105455.8+580041,163.732324,58.011442,148.299496,52.996053,5.036216,70.531242,1.390370e-14,2.112673e-15,65.903633,10.014083,4330.384766,0.0,0.0,0.0,2550,2020,163.734333,58.010152,2.085002,129.567734,0.000012,6.268413e+08,6.270701e+08,0.0,0.0,0.015219,0.002313,17.583460,163.732324,58.011442,-1,-1.000000,0,-1,NaN,-1.0,-1,NaN,NaN,NaN,-1.000000,0,1,0,0,16.599001,15.829,12.399,15.0,9.2,0.9,1,1,0,1237658303279792699,0,4.868471,62608417819,2.085002,1.800482,2.717993,4.407123,137.041885,45

## CSC x DESI cross-match

In [27]:
csc_desi_all = cross_match_data_frames(ero_csc, desi, colname_ra1 = 'csc_ra',colname_dec1 = 'csc_dec' , colname_ra2 = 'ra', colname_dec2=  'dec', match_radius = 15, df_prefix = 'desi')
csc_desi_all

cross-match radius 15 arcsec
total matches: 2093 out of 461 x 2418574
	 total unique pairs: 2093
	 total non-unique pairs (duplicates in df2): 0


,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH,csc_name,csc_ra,csc_dec,csc_err_ellipse_r0,csc_err_ellipse_r1,csc_err_ellipse_ang,csc_significance,csc_likelihood,csc_likelihood_class,csc_conf_flag,csc_dither_warning_flag,csc_extent_flag,csc_pileup_flag,csc_sat_src_flag,csc_streak_src_flag,csc_var_flag,csc_flux_aper_s,csc_flux_aper_lolim_s,csc_flux_aper_hilim_s,csc_flux_aper_m,csc_flux_aper_lolim_m,csc_flux_aper_hilim_m,csc_radec_err,csc_r_98,csc_flux_05_2,csc_flux_05_2_err,csc_secure,csc_sep,csc_n_near,csc_n_matches,desi_release,desi_objid,desi_brickid,desi_ra,desi_dec,desi_flux_g,desi_flux_r,desi_flux_z,desi_flux_w1,desi_flux_w2,desi_flux_w3,desi_flux_w4,desi_flux_ivar_g,desi_flux_ivar_r,desi_flux_ivar_z,desi_flux_ivar_w1,desi_flux_ivar_w2,desi_flux_ivar_w3,desi_flux_ivar_w4,desi_dered_mag_g,desi_dered_mag_r,desi_dered_mag_z,desi_dered_mag_w1,desi_dered_mag_w2,desi_dered_mag_w3,desi_dered_mag_w4,desi_snr_g,desi_snr_r,desi_snr_z,desi_snr_w1,desi_snr_w2,desi_snr_w3,desi_snr_w4,desi_type,desi_parallax,desi_parallax_ivar,desi_pmra,desi_pmra_ivar,desi_pmdec,desi_pmdec_ivar,desi_ref_cat,desi_ref_id,desi_mjd_max,desi_mjd_min,desi_iso_max,desi_iso_min,desi_desi_id,desi_sep,desi_n_near,desi_n_matches
2,SRGe J110242.6+594120,165.677384,59.688978,145.179791,52.486946,7.653812,211.181183,1.846204e-13,1.864554e-14,134.683884,13.602254,666.474426,0.0,0.0,0.0,679,556,165.679392,59.687689,3.316714,224.252457,0.000026,6.268413e+08,6.270701e+08,0.0,0.0,0.202084,0.020409,3.928642,165.677384,59.688978,860692071522063104,4.485168,1,0,17.027691,-1.0,0,6C 105938+595728,1.830,BLLac,4.485268,1,1,1,1,13.617,12.659,9.803,43.9,47.2,27.9,1,3,1,1237658304891060264,0,3.087692,56364004534,3.316714,2.736290,4.130682,6.697745,136.878443,47.936061,1.420726e-13,6.398279e+19,2CXO J110242.8+594122,165.678600,59.689447,2.291189,2.008557,31.648612,14.313177,629.754672,TRUE,False,False,False,False,False,False,False,1.247590e-13,1.097446e-13,1.397733e-13,9.484644e-14,8.480775e-14,1.048851e-13,1.244794,3.481875,2.196054e-13,2.554235e-14,True,2.779278,1,1,9011,4152,616737,165.678178,59.688774,123.783880,140.066150,194.811190,287.176500,388.446700,1089.463000,2591.550300,53.469475,22.336110,17.090487,1.092097,0.326052,0.001556,0.000019,17.247953,17.120436,16.768284,16.353460,16.025955,14.906816,13.966043,905.142700,661.967900,805.361940,300.109280,221.806760,42.980423,11.174195,PSF,-0.023313,140.88054,-0.008907,94.32313,0.07499,50.308243,G2,860692071522063104,58194.477145,57813.360879,2018-03-17 11:27:05.369,2017-03-01 08:39:39.978,9011_616737_4152,2.540902,7,1
1,SRGe J110242.6+594120,165.677384,59.688978,145.179791,52.486946,7.653812,211.181183,1.846204e-13,1.864554e-14,134.683884,13.602254,666.474426,0.0,0.0,0.0,679,556,165.679392,59.687689,3.316714,224.252457,0.000026,6.268413e+08,6.270701e+08,0.0,0.0,0.202084,0.020409,3.928642,165.677384,59.688978,860692071522063104,4.485168,1,0,17.027691,-1.0,0,6C 105938+595728,1.830,BLLac,4.485268,1,1,1,1,13.617,12.659,9.803,43.9,47.2,27.9,1,3,1,1237658304891060264,0,3.087692,56364004534,3.316714,2.736290,4.130682,6.697745,136.878443,47.936061,1.420726e-13,6.398279e+19,2CXO J110242.8+594122,165.678600,59.689447,2.291189,2.008557,31.648612,14.313177,629.754672,TRUE,False,False,False,False,False,False,False,1.247590e-13,1.097446e-13,1.397733e-13,9.484644e-14,8.480775e-14,1.048851e-13,1.244794,3.481875,2.196054e-13,2.554235e-14,True,2.779278,1,1,9011,4143,616737,165.676791,59.691063,-0.015934,0.072536,0.596225,3.677433,6.544262,-131.121550,-1138.750900,749.922000,238.

In [28]:
csc_desi_closest = (csc_desi_all
    .loc[csc_desi_all.groupby('csc_name')['desi_sep'].idxmin()]
    )

csc_hostless = csc_desi_closest.query("desi_sep>2*csc_r_98 & desi_sep>2*1.47")

csc_hostless['csc_ero_flux_ratio'] = csc_hostless['csc_flux_05_2'] / csc_hostless['flux_05-20']


csc_hostless.rename(columns = {'csc_sep':'ero_csc_sep', 'desi_sep':'desi_sep_minimal'}, inplace = True)

csc_hostless = add_separation_columns(csc_hostless, 'RA_fin', 'DEC_fin', 'desi_ra', 'desi_dec', 'desi_ero_sep')

assert csc_hostless.srcname_fin.nunique()==csc_hostless.csc_name.nunique()
assert csc_hostless.srcname_fin.nunique()==csc_hostless.shape[0]


print('hostless csc near chandra sources', csc_hostless.shape[0])


csc_hostless.sample(7)

hostless csc near chandra sources 13


,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH,csc_name,csc_ra,csc_dec,csc_err_ellipse_r0,csc_err_ellipse_r1,csc_err_ellipse_ang,csc_significance,csc_likelihood,csc_likelihood_class,csc_conf_flag,csc_dither_warning_flag,csc_extent_flag,csc_pileup_flag,csc_sat_src_flag,csc_streak_src_flag,csc_var_flag,csc_flux_aper_s,csc_flux_aper_lolim_s,csc_flux_aper_hilim_s,csc_flux_aper_m,csc_flux_aper_lolim_m,csc_flux_aper_hilim_m,csc_radec_err,csc_r_98,csc_flux_05_2,csc_flux_05_2_err,csc_secure,ero_csc_sep,csc_n_near,csc_n_matches,desi_release,desi_objid,desi_brickid,desi_ra,desi_dec,desi_flux_g,desi_flux_r,desi_flux_z,desi_flux_w1,desi_flux_w2,desi_flux_w3,desi_flux_w4,desi_flux_ivar_g,desi_flux_ivar_r,desi_flux_ivar_z,desi_flux_ivar_w1,desi_flux_ivar_w2,desi_flux_ivar_w3,desi_flux_ivar_w4,desi_dered_mag_g,desi_dered_mag_r,desi_dered_mag_z,desi_dered_mag_w1,desi_dered_mag_w2,desi_dered_mag_w3,desi_dered_mag_w4,desi_snr_g,desi_snr_r,desi_snr_z,desi_snr_w1,desi_snr_w2,desi_snr_w3,desi_snr_w4,desi_type,desi_parallax,desi_parallax_ivar,desi_pmra,desi_pmra_ivar,desi_pmdec,desi_pmdec_ivar,desi_ref_cat,desi_ref_id,desi_mjd_max,desi_mjd_min,desi_iso_max,desi_iso_min,desi_desi_id,desi_sep_minimal,desi_n_near,desi_n_matches,csc_ero_flux_ratio,desi_ero_sep
1680,SRGe J103409.4+574725,158.539363,57.790228,151.484635,51.041462,7.180546,17.755358,5.841617e-15,1.528532e-15,27.442596,7.180698,4291.801758,0.0,0.0,0.0,6796,5693,158.541372,57.788938,3.097480,164.290009,0.000004,6.268413e+08,6.270701e+08,0.0,0.0,0.006394,0.001673,18.891672,158.539363,57.790228,-1,-1.0,0,-1,NaN,-1.0,-1,CXCCLASXS J103409.5+574728,1.350,X,8.683937,1,1,0,0,18.027000,17.357000,12.493,5.0,2.4,1.1,1,0,-1,0,0,NaN,63454851294,3.097480,2.567094,3.875265,6.283596,133.900475,44.323787,4.495352e-15,5.647276e+19,2CXO J103409.5+574728,158.539889,57.791195,0.772872,0.749767,165.551208,5.682467,250.406246,TRUE,False,False,False,False,False,False,False,1.084245e-15,7.160107e-16,1.432021e-15,1.102562e-15,7.603873e-16,1.425726e-15,0.439912,1.230499,2.186806e-15,6.914214e-16,True,3.623997,1,1,9011,3565,610715,158.543036,57.792069,0.342256,0.550325,0.474627,1.722385,2.746398,3.253695,812.706700,369.84216,96.478080,46.181720,4.564278,1.094711,0.001548,0.000019,23.649763,23.138779,23.303707,21.908852,21.402586,21.218950,15.225124,6.582026,5.405471,3.225432,3.679732,2.873513,0.128028,3.572985,REX,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,58190.292931,57820.346481,2018-03-13 07:01:49.231,2017-03-08 08:18:56.000,9011_610715_3565,6.808410,6,1,0.486459,9.673252
1515,SRGe J104544.5+585020,161.435576,58.838784,148.644979,51.551852,6.675908,22.572021,6.524672e-15,1.522000e-15,30.522713,7.119986,4273.777832,0.0,0.0,0.0,5546,4550,161.437585,58.837494,2.862482,176.957397,0.000008,6.268413e+08,6.270701e+08,0.0,0.0,0.007142,0.001666,18.182652,161.435576,58.838784,-1,-1.0,0,-1,NaN,-1.0,-1,CLANS 363,0.650,Galaxy,7.376943,1,1,0,0,16.285999,16.346001,12.797,16.9,5.6,-0.3,1,1,0,1237655109446468260,0,9.526234,59491076346,2.862482,2.386682,3.602917,5.841994,134.941256,46.019237,5.020989e-15,8.017007e+19,2CXO J104544.2+585019,161.434203,58.838799,1.391252,1.131932,35.436774,4.666667,63.731146,TRUE,False,False,False,False,False,False,False,2.118220e-15,1.370613e-15,2.865827e-15,2.012019e-15,1.516753e-15,2.476331e-15,0.732739,2.049579,4.130239e-15,1.256466e-15,True,2.557606,1,1,9011,1041,613768,161.428911,58.837392,0.164835,0.411761,2.994679,13.263495,9.807838,5.826660,-125.004380,431.63490,121.841870,47.155907,4.137182,1.078248,0.001186,0.000014,24.431760,23.446133

***

# 4XMM DR10

In [29]:
xmm_df = pd.read_pickle(data_path+'xmm_all_lh.pkl')
xmm_df.sample(5)

,srcid,iauname,sc_ra,sc_dec,sc_poserr,sc_det_ml,sc_ep_1_flux,sc_ep_1_flux_err,sc_ep_2_flux,sc_ep_2_flux_err,sc_ep_3_flux,sc_ep_3_flux_err,sc_ep_4_flux,sc_ep_4_flux_err,sc_ep_5_flux,sc_ep_5_flux_err,sc_ep_8_flux,sc_ep_8_flux_err,sc_ep_9_flux,sc_ep_9_flux_err,sc_ep_8_fmin,sc_ep_8_fmin_err,sc_ep_8_fmax,sc_ep_8_fmax_err,sc_hr1,sc_hr1_err,sc_hr2,sc_hr2_err,sc_hr3,sc_hr3_err,sc_hr4,sc_hr4_err,sc_extent,sc_ext_err,sc_ext_ml,sc_chi2prob,sc_var_flag,sc_sum_flag,mjd_first,mjd_last,sc_fvar,sc_fvarerr,n_detections,confused,webpage_url,flux_05_2,flux_05_2_err,sigma,r_98,secure
246805,203032602010059,4XMM J103449.8+574654,158.707864,57.781692,1.45453,47.12750,6.154570e-17,1.991560e-16,6.966320e-16,4.110740e-16,2.394240e-15,7.926540e-16,5.225600e-15,1.806320e-15,1.032440e-14,1.006020e-14,2.567740e-14,1.036600e-14,9.359160e-15,1.576740e-15,1.849310e-14,1.069110e-14,1.384680e-13,4.236070e-14,0.258761,0.189439,0.459801,0.117138,-0.185362,0.126230,-0.582888,0.223238,67.437,4.7342,48.486300,0.00547,False,1,53467.914178,53510.606285,6.243419,1.770275,2,False,http://xmm-catalog.irap.omp.eu/source/20303260...,3.090872e-15,8.929066e-16,1.031477,2.885194,False
247880,205562109015025,4XMM J103821.7+575436,159.590655,57.910147,1.90296,8.73683,1.504320e-15,9.233980e-16,6.493530e-16,7.354910e-16,2.036620e-15,1.208320e-15,7.569200e-15,3.914100e-15,2.519850e-15,9.596440e-15,1.786950e-14,1.120140e-14,9.139630e-15,3.007120e-15,1.786950e-14,1.120140e-14,1.786950e-14,1.120140e-14,0.146089,0.368641,0.066447,0.335760,0.128209,0.382978,-0.051405,0.384495,0.000,NaN,-0.543932,NaN,NaN,0,54619.549190,54619.630590,NaN,NaN,1,False,http://xmm-catalog.irap.omp.eu/source/20556210...,2.685973e-15,1.414561e-15,1.349480,3.774697,False
251817,205541201010096,4XMM J104521.2+585047,161.338494,58.846529,2.11267,9.18941,2.963210e-16,2.347530e-16,2.201450e-16,2.272350e-16,6.238010e-16,4.036580e-16,1.273210e-15,8.991900e-16,5.157100e-16,3.931650e-15,4.445610e-15,4.163290e-15,2.166060e-15,8.716960e-16,4.445610e-15,4.163290e-15,4.445610e-15,4.163290e-15,0.113628,0.212730,0.173191,0.233079,-0.038369,0.350912,-0.429134,0.619208,0.000,NaN,1.716080,NaN,NaN,0,54750.880949,54751.340602,NaN,NaN,1,False,http://xmm-catalog.irap.omp.eu/source/20554120...,8.439460e-16,4.632230e-16,1.498195,4.190676,False
249725,205562112010031,4XMM J104137.4+590807,160.406070,59.135412,1.85697,8.34118,1.574060e-17,1.671250e-16,1.388020e-16,3.158600e-16,4.861030e-17,3.081620e-16,0.000000e+00,9.060370e-16,2.056990e-15,7.984670e-15,3.111350e-15,8.056740e-15,1.645060e-16,4.522680e-16,3.111350e-15,8.056740e-15,3.111350e-15,8.056740e-15,-0.108644,0.333161,-0.230160,0.336849,-1.000000,0.448224,1.000000,0.540079,0.000,NaN,-0.532336,NaN,NaN,0,54595.266366,54595.352928,NaN,NaN,1,False,http://xmm-catalog.irap.omp.eu/source/20556211...,1.874123e-16,4.412838e-16,1.316866,3.683471,False
246275,203060507010039,4XMM J103333.3+600243,158.389089,60.045449,1.79866,15.51190,4.804630e-16,4.217750e-16,4.612940e-16,3.618320e-16,3.029510e-16,2.800820e-16,2.515930e-15,1.205560e-15,7.887730e-15,5.652240e-15,1.415460e-14,5.919320e-15,3.622020e-15,1.042080e-15,1.415460e-14,5.919320e-15,1.415460e-14,5.919320e-15,-0.055058,0.454645,0.339908,0.230252,-0.255377,0.231709,-0.226501,0.358343,0.000,NaN,-0.700996,NaN,NaN,0,53464.316447,53464.431447,NaN,NaN,1,False,http://xmm-catalog.irap.omp.eu/source/20306050...,7.642450e-16,4.575678e-16,1.275516,3.567808,True


## XMM x eROSITA cross-match

In [30]:
ero_xmm_orig = cross_match_data_frames(ero_df, xmm_df,
 'RA_fin', 'DEC_fin', 'sc_ra', 'sc_dec', match_radius = 30, df_prefix = 'xmm')
 
ero_xmm = ero_xmm_orig.query("xmm_n_near==1 & xmm_n_matches==1")
ero_xmm = ero_xmm.query("xmm_secure==True")
print('SECURITY FILTER APPLIED (XMM)')

ero_xmm = ero_xmm.query("xmm_secure==True")
print('cross-matches: only one xmm within 30 arcsec and it is unique for the eROSITA source', ero_xmm.shape[0])
assert len(ero_xmm) == ero_xmm.xmm_srcid.nunique()
ero_xmm.sample(5)

cross-match radius 30 arcsec
total matches: 839 out of 6885 x 1791
	 total unique pairs: 817
	 total non-unique pairs (duplicates in df2): 22
SECURITY FILTER APPLIED (XMM)
cross-matches: only one xmm within 30 arcsec and it is unique for the eROSITA source 646


,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH,xmm_srcid,xmm_iauname,xmm_sc_ra,xmm_sc_dec,xmm_sc_poserr,xmm_sc_det_ml,xmm_sc_ep_1_flux,xmm_sc_ep_1_flux_err,xmm_sc_ep_2_flux,xmm_sc_ep_2_flux_err,xmm_sc_ep_3_flux,xmm_sc_ep_3_flux_err,xmm_sc_ep_4_flux,xmm_sc_ep_4_flux_err,xmm_sc_ep_5_flux,xmm_sc_ep_5_flux_err,xmm_sc_ep_8_flux,xmm_sc_ep_8_flux_err,xmm_sc_ep_9_flux,xmm_sc_ep_9_flux_err,xmm_sc_ep_8_fmin,xmm_sc_ep_8_fmin_err,xmm_sc_ep_8_fmax,xmm_sc_ep_8_fmax_err,xmm_sc_hr1,xmm_sc_hr1_err,xmm_sc_hr2,xmm_sc_hr2_err,xmm_sc_hr3,xmm_sc_hr3_err,xmm_sc_hr4,xmm_sc_hr4_err,xmm_sc_extent,xmm_sc_ext_err,xmm_sc_ext_ml,xmm_sc_chi2prob,xmm_sc_var_flag,xmm_sc_sum_flag,xmm_mjd_first,xmm_mjd_last,xmm_sc_fvar,xmm_sc_fvarerr,xmm_n_detections,xmm_confused,xmm_webpage_url,xmm_flux_05_2,xmm_flux_05_2_err,xmm_sigma,xmm_r_98,xmm_secure,xmm_sep,xmm_n_near,xmm_n_matches
365,SRGe J104559.6+590630,161.498517,59.108445,148.287967,51.395902,5.317735,57.893604,1.310871e-14,2.102448e-15,62.233639,9.981380,4337.233887,0.0,0.0,0.0,1068,861,161.500525,59.107156,2.220601,45.433483,0.000005,6.268413e+08,6.270701e+08,0.0,0.0,0.014349,0.002301,17.868204,161.498517,59.108445,855361978451959296,6.469386,1,0,19.604927,-1.0,0,3XMM J104559.6+590631,2.513000,QSO,6.467511,3,1,1,1,16.638000,15.550000,12.253,13.4,10.3,3.0,1,1,1,1237658304890077373,7988437246448783360,6.506487,58490857182,2.220601,1.901127,2.869926,4.653476,134.768690,46.263108,1.008766e-14,8.325804e+19,205541201010009,4XMM J104559.6+590631,161.498716,59.108623,0.361928,604.8780,6.052030e-15,4.331820e-16,7.362660e-15,4.843560e-16,6.645420e-15,5.302590e-16,8.793780e-15,1.098660e-15,7.760260e-15,3.950520e-15,3.875530e-14,4.299770e-15,2.459390e-14,1.172950e-15,3.164160e-14,9.775290e-15,5.866440e-14,1.195850e-14,0.113673,0.046615,-0.110611,0.047641,-0.402685,0.058932,-0.165464,0.129768,0.0,NaN,-1.46687,0.006894,False,0,54750.880949,54932.883727,0.678534,0.196583,4,False,http://xmm-catalog.irap.omp.eu/source/20554120...,1.400808e-14,7.181750e-16,0.256660,0.717918,True,0.739397,1,1
101,SRGe J104818.1+572425,162.075332,57.406907,150.029275,52.753162,3.251884,265.094818,3.437605e-14,3.068333e-15,160.832947,14.355606,4274.314941,0.0,0.0,0.0,565,469,162.077341,57.405617,1.176899,106.866035,0.000010,6.268413e+08,6.270701e+08,0.0,0.0,0.037628,0.003359,19.042423,162.075332,57.406907,854226767056200576,4.566791,1,0,20.194593,-1.0,0,SDSS J104818.20+572424.1,0.684000,QSO,4.567821,2,1,1,0,15.928000,15.231000,11.584,21.8,13.9,5.3,1,3,1,1237658302742462720,0,3.827304,64933232563,1.176899,1.162571,1.755008,2.845679,136.448766,44.995348,2.645371e-14,6.640979e+19,205562116010012,4XMM J104818.2+572423,162.075837,57.406518,1.057790,112.7550,1.539880e-14,3.951060e-15,2.424830e-14,5.057240e-15,2.111320e-14,4.513280e-15,1.639130e-14,7.088840e-15,4.949200e-15,3.007910e-14,8.730280e-14,3.204080e-14,7.040410e-14,1.015980e-14,8.730280e-14,3.204080e-14,8.730280e-14,3.204080e-14,0.175174,0.152696,0.024482,0.146841,-0.556568,0.162547,-0.888940,0.622142,0.0,NaN,-1.15832,NaN,NaN,0,54799.044537,54799.125949,NaN,NaN,1,False,http://xmm-catalog.irap.omp.eu/source/20556211...,4.536150e-14,6.778302e-15,0.750129,2.098224,True,1.708372,1,1
406,SRGe J104524.4+590704,161.351746,59.117719,148.357638,51.333689,4.781737,59.700226,1.161651e-14,1.888342e-15,54.700184,8.891879,4301.901367,0.0,0.0,0.0,1613,1271,161.353755,59.116429,1.961326,50.090984,0.000006,6.268413e+08,6.270701e+08,0.0,0.0,0.012715,0.002067,17.312874,161.351746,59.117719,855363833877758336,6.141886,1,1,11.691483,NaN,0,Gaia DR2 855

## XMM x DESI cross-match

In [31]:
xmm_desi_all = cross_match_data_frames(ero_xmm, desi, colname_ra1 = 'xmm_sc_ra',colname_dec1 = 'xmm_sc_dec' , colname_ra2 = 'ra', colname_dec2=  'dec', match_radius = 15, df_prefix = 'desi')
xmm_desi_all

cross-match radius 15 arcsec
total matches: 2831 out of 646 x 2418574
	 total unique pairs: 2831
	 total non-unique pairs (duplicates in df2): 0


,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH,xmm_srcid,xmm_iauname,xmm_sc_ra,xmm_sc_dec,xmm_sc_poserr,xmm_sc_det_ml,xmm_sc_ep_1_flux,xmm_sc_ep_1_flux_err,xmm_sc_ep_2_flux,xmm_sc_ep_2_flux_err,xmm_sc_ep_3_flux,xmm_sc_ep_3_flux_err,xmm_sc_ep_4_flux,xmm_sc_ep_4_flux_err,xmm_sc_ep_5_flux,xmm_sc_ep_5_flux_err,xmm_sc_ep_8_flux,xmm_sc_ep_8_flux_err,xmm_sc_ep_9_flux,xmm_sc_ep_9_flux_err,xmm_sc_ep_8_fmin,xmm_sc_ep_8_fmin_err,xmm_sc_ep_8_fmax,xmm_sc_ep_8_fmax_err,xmm_sc_hr1,xmm_sc_hr1_err,xmm_sc_hr2,xmm_sc_hr2_err,xmm_sc_hr3,xmm_sc_hr3_err,xmm_sc_hr4,xmm_sc_hr4_err,xmm_sc_extent,xmm_sc_ext_err,xmm_sc_ext_ml,xmm_sc_chi2prob,xmm_sc_var_flag,xmm_sc_sum_flag,xmm_mjd_first,xmm_mjd_last,xmm_sc_fvar,xmm_sc_fvarerr,xmm_n_detections,xmm_confused,xmm_webpage_url,xmm_flux_05_2,xmm_flux_05_2_err,xmm_sigma,xmm_r_98,xmm_secure,xmm_sep,xmm_n_near,xmm_n_matches,desi_release,desi_objid,desi_brickid,desi_ra,desi_dec,desi_flux_g,desi_flux_r,desi_flux_z,desi_flux_w1,desi_flux_w2,desi_flux_w3,desi_flux_w4,desi_flux_ivar_g,desi_flux_ivar_r,desi_flux_ivar_z,desi_flux_ivar_w1,desi_flux_ivar_w2,desi_flux_ivar_w3,desi_flux_ivar_w4,desi_dered_mag_g,desi_dered_mag_r,desi_dered_mag_z,desi_dered_mag_w1,desi_dered_mag_w2,desi_dered_mag_w3,desi_dered_mag_w4,desi_snr_g,desi_snr_r,desi_snr_z,desi_snr_w1,desi_snr_w2,desi_snr_w3,desi_snr_w4,desi_type,desi_parallax,desi_parallax_ivar,desi_pmra,desi_pmra_ivar,desi_pmdec,desi_pmdec_ivar,desi_ref_cat,desi_ref_id,desi_mjd_max,desi_mjd_min,desi_iso_max,desi_iso_min,desi_desi_id,desi_sep,desi_n_near,desi_n_matches
5,SRGe J104117.7+584929,160.323629,58.824642,149.266632,51.126914,2.166339,3391.888916,2.361506e-13,7.471780e-15,1120.011108,35.437035,4332.920898,0.0,0.0,0.0,38,27,160.325638,58.823353,0.454968,97.643723,0.000006,6.268413e+08,6.270701e+08,0.0,0.0,0.258489,0.008179,18.102430,160.323629,58.824642,-1,-1.00000,0,-1,NaN,-1.0,-1,NaN,NaN,NaN,-1.000000,0,1,1,0,15.243,15.129,12.821,31.5,15.9,-0.2,1,3,0,1237658304889815429,0,2.181367,59543516327,0.454968,0.774481,1.169150,1.895733,134.256532,45.696772,1.817271e-13,7.704875e+19,205562112010003,4XMM J104117.9+584929,160.324685,58.824762,0.817968,742.578,2.808530e-14,3.382310e-15,4.008580e-14,4.210150e-15,4.761820e-14,4.978040e-15,5.426170e-14,9.520310e-15,8.843220e-15,2.233210e-14,1.909300e-13,2.733320e-14,1.567980e-13,1.062820e-14,1.909300e-13,2.733320e-14,1.909300e-13,2.733320e-14,0.136026,0.075853,0.023020,0.072849,-0.437121,0.081245,-0.896701,0.157718,0.0,NaN,-2.276850,0.425014,False,0,54595.266366,54595.352928,0.012697,1.426367,1,False,http://xmm-catalog.irap.omp.eu/source/20556211...,8.770400e-14,6.519681e-15,0.580060,1.622515,True,2.015272,1,1,9011,43,613766,160.324219,58.824639,-0.052349,0.298325,1.900912,2.352417,2.387243,53.370483,2019.36580,671.00964,212.87097,70.456120,2.689567,0.915560,0.001440,0.000018,NaN,23.799402,21.794834,21.570036,21.554535,18.181593,14.236904,-1.356040,4.352592,15.955904,3.857940,2.284231,2.025409,8.467377,PSF,0.00000,0.00000,0.000000,0.00000,0.000000,0.00000,NaN,0,58194.398524,57851.315637,2018-03-17 09:33:52.450,2017-04-08 07:34:31,9011_613766_43,0.975167,9,1
6,SRGe J104117.7+584929,160.323629,58.824642,149.266632,51.126914,2.166339,3391.888916,2.361506e-13,7.471780e-15,1120.011108,35.437035,4332.920898,0.0,0.0,0.0,38,27,160.325638,58.823353,0.454968,97.643723,0.000006,6.268413e+08,6.270701e+08,0.0,0.0,0.258489,0.008179,18.102430,160.323629,58.824642,-1,-1.00000,0,-1,NaN,-1.0,-1,NaN,NaN,NaN,-1.000000,0,1,1,0,15.243,15.129,12.821,31.5,15.9,-0.2,1,3,0,1237658304889815429,0,2

In [32]:
xmm_desi_closest = (xmm_desi_all
    .loc[xmm_desi_all.groupby('xmm_srcid')['desi_sep'].idxmin()]
    )

xmm_hostless = xmm_desi_closest.query("desi_sep>2*xmm_r_98 & desi_sep>2*1.47")
#xmm_hostless = xmm_hostless.query('xmm_sep<20') #why we have added this in the first place?

xmm_hostless['xmm_ero_flux_ratio'] = xmm_hostless['xmm_flux_05_2'] / xmm_hostless['flux_05-20']


xmm_hostless.rename(columns = {'xmm_sep':'ero_xmm_sep', 'desi_sep':'desi_sep_minimal'}, inplace = True)

xmm_hostless = add_separation_columns(xmm_hostless, 'RA_fin', 'DEC_fin', 'desi_ra', 'desi_dec', 'desi_ero_sep')

assert xmm_hostless.srcname_fin.nunique()==xmm_hostless.xmm_srcid.nunique()
assert xmm_hostless.srcname_fin.nunique()==xmm_hostless.shape[0]


print('hostless xmm near chandra sources', xmm_hostless.shape[0])


xmm_hostless.sample(7)

hostless xmm near chandra sources 23


,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH,xmm_srcid,xmm_iauname,xmm_sc_ra,xmm_sc_dec,xmm_sc_poserr,xmm_sc_det_ml,xmm_sc_ep_1_flux,xmm_sc_ep_1_flux_err,xmm_sc_ep_2_flux,xmm_sc_ep_2_flux_err,xmm_sc_ep_3_flux,xmm_sc_ep_3_flux_err,xmm_sc_ep_4_flux,xmm_sc_ep_4_flux_err,xmm_sc_ep_5_flux,xmm_sc_ep_5_flux_err,xmm_sc_ep_8_flux,xmm_sc_ep_8_flux_err,xmm_sc_ep_9_flux,xmm_sc_ep_9_flux_err,xmm_sc_ep_8_fmin,xmm_sc_ep_8_fmin_err,xmm_sc_ep_8_fmax,xmm_sc_ep_8_fmax_err,xmm_sc_hr1,xmm_sc_hr1_err,xmm_sc_hr2,xmm_sc_hr2_err,xmm_sc_hr3,xmm_sc_hr3_err,xmm_sc_hr4,xmm_sc_hr4_err,xmm_sc_extent,xmm_sc_ext_err,xmm_sc_ext_ml,xmm_sc_chi2prob,xmm_sc_var_flag,xmm_sc_sum_flag,xmm_mjd_first,xmm_mjd_last,xmm_sc_fvar,xmm_sc_fvarerr,xmm_n_detections,xmm_confused,xmm_webpage_url,xmm_flux_05_2,xmm_flux_05_2_err,xmm_sigma,xmm_r_98,xmm_secure,ero_xmm_sep,xmm_n_near,xmm_n_matches,desi_release,desi_objid,desi_brickid,desi_ra,desi_dec,desi_flux_g,desi_flux_r,desi_flux_z,desi_flux_w1,desi_flux_w2,desi_flux_w3,desi_flux_w4,desi_flux_ivar_g,desi_flux_ivar_r,desi_flux_ivar_z,desi_flux_ivar_w1,desi_flux_ivar_w2,desi_flux_ivar_w3,desi_flux_ivar_w4,desi_dered_mag_g,desi_dered_mag_r,desi_dered_mag_z,desi_dered_mag_w1,desi_dered_mag_w2,desi_dered_mag_w3,desi_dered_mag_w4,desi_snr_g,desi_snr_r,desi_snr_z,desi_snr_w1,desi_snr_w2,desi_snr_w3,desi_snr_w4,desi_type,desi_parallax,desi_parallax_ivar,desi_pmra,desi_pmra_ivar,desi_pmdec,desi_pmdec_ivar,desi_ref_cat,desi_ref_id,desi_mjd_max,desi_mjd_min,desi_iso_max,desi_iso_min,desi_desi_id,desi_sep_minimal,desi_n_near,desi_n_matches,xmm_ero_flux_ratio,desi_ero_sep
1478,SRGe J103518.6+562405,158.827436,56.401399,153.128304,51.994021,6.233959,43.079376,1.132047e-14,1.976916e-15,53.370865,9.320259,4307.122070,0.0,0.0,0.0,2626,2089,158.829445,56.400109,2.655373,125.111130,0.000007,6.268413e+08,6.270701e+08,0.0,0.0,0.012391,0.002164,19.447279,158.827436,56.401399,-1,-1.0,0,-1,NaN,-1.0,-1,NaN,NaN,NaN,-1.000000,0,1,0,-1,NaN,NaN,NaN,NaN,NaN,NaN,0,2,0,1237655107298066604,0,4.348417,68884283132,2.655373,2.228683,3.364402,5.455250,135.073457,43.212077,8.711545e-15,5.354746e+19,202025203010014,4XMM J103518.0+562408,158.825121,56.402300,1.527970,27.2121,5.272640e-16,8.734870e-16,3.118090e-15,1.320410e-15,6.652110e-15,1.823750e-15,1.177980e-14,4.244750e-15,0.000000e+00,9.133890e-15,2.328970e-14,1.056570e-14,1.995850e-14,3.850720e-15,2.328970e-14,1.056570e-14,2.328970e-14,1.056570e-14,0.724272,0.296010,0.306399,0.215377,-0.221640,0.212824,-1.000000,0.313114,0.0,NaN,-0.878471,NaN,NaN,0,53299.251389,53299.570845,NaN,NaN,1,False,http://xmm-catalog.irap.omp.eu/source/20202520...,9.770200e-15,2.251565e-15,1.083556,3.030869,True,5.638750,1,1,9011,302,606788,158.827919,56.401126,0.319972,0.322629,0.555002,2.123019,1.617711,55.201880,-39.081413,696.00433,189.474300,56.964806,4.602108,1.119031,0.001178,0.000014,23.715626,23.713694,23.131126,21.681380,21.976988,18.144953,NaN,8.441467,4.440982,4.188881,4.554413,1.711283,1.894542,-0.148242,REX,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,58174.311153,57518.163021,2018-02-25 07:28:03.630,2016-05-10 03:54:45.000,9011_606788_302,6.995479,5,1,1.121523,1.375083
2540,SRGe J104122.8+590252,160.344824,59.047725,148.985124,50.990605,8.902710,13.570358,5.498920e-15,1.535318e-15,25.644791,7.160118,4260.585449,0.0,0.0,0.0,6102,5048,160.346833,59.046436,3.891301,45.830978,0.000008,6.268413e+08,6.270701e+08,0.0,0.0,0.006019,0.001681,37.311096,160.344824,59.047725,-1,-1.0,0,-1,NaN,-1.0,-1,NaN,NaN,NaN,-1.000000,0,1,0,1,17.937000,16.915001,12.550,5.1,1.4,0.1,1,0,-1,0,0,NaN,5

## Cross-correlate eROSITA with XMM and Chandra

so that for every eROSITA source we have a number of neighbors in XMM and Chandra. 

In [33]:
ero_csc_n_neigh = cross_match_data_frames(ero_df, csc_df,
 'RA_fin', 'DEC_fin', 'ra', 'dec', match_radius = 30, df_prefix = 'csc')
n_chandra_dict = dict(ero_csc_n_neigh.groupby('srcname_fin')['csc_sep'].count())

ero_xmm_n_neigh = cross_match_data_frames(ero_df, xmm_df,
    'RA_fin', 'DEC_fin', 'sc_ra', 'sc_dec', match_radius = 30, df_prefix = 'xmm')
n_xmm_dict = dict(ero_xmm_n_neigh.groupby('srcname_fin')['xmm_sep'].count())

cross-match radius 30 arcsec
total matches: 777 out of 6885 x 2329
	 total unique pairs: 722
	 total non-unique pairs (duplicates in df2): 55
cross-match radius 30 arcsec
total matches: 839 out of 6885 x 1791
	 total unique pairs: 817
	 total non-unique pairs (duplicates in df2): 22


# Joining CSC and XMM counterparts

In [34]:
csc_ctps = csc_hostless[['srcname_fin', 'RA_fin','DEC_fin', 'DET_LIKE_0', 'pos_r98', 'pos_sigma_2d', 'flux_05-20', 'ero_csc_sep', 'csc_name',   'csc_ra', 'csc_dec', 'csc_flux_05_2', 'csc_r_98', 'desi_sep_minimal', 'desi_desi_id',  'desi_ra', 'desi_dec', 'csc_ero_flux_ratio']]

xmm_ctps = xmm_hostless[['srcname_fin', 'RA_fin','DEC_fin', 'DET_LIKE_0', 'pos_r98', 'pos_sigma_2d', 'flux_05-20', 'ero_xmm_sep', 'xmm_srcid',   'xmm_sc_ra', 'xmm_sc_dec', 'xmm_flux_05_2', 'xmm_r_98', 'desi_sep_minimal', 'desi_desi_id',  'desi_ra', 'desi_dec', 'xmm_ero_flux_ratio']]

csc_ctps.rename(columns = {'desi_desi_id':'desi_id'}, inplace = True)
xmm_ctps.rename(columns = {'desi_desi_id':'desi_id'}, inplace = True)

In [35]:
print('XMM companions', xmm_ctps.shape[0])
print('CSC companions', csc_ctps.shape[0])
print('Total companions possible', xmm_ctps.shape[0] + csc_ctps.shape[0])

XMM companions 23
CSC companions 13
Total companions possible 36


In [36]:
xmm_ctps = xmm_ctps[xmm_ctps['xmm_ero_flux_ratio'] <  5 ]
xmm_ctps = xmm_ctps[xmm_ctps['xmm_ero_flux_ratio'] >  1/5 ]

csc_ctps = csc_ctps[csc_ctps['csc_ero_flux_ratio'] <  5 ]
csc_ctps = csc_ctps[csc_ctps['csc_ero_flux_ratio'] >  1/5 ]

In [37]:
xmm_ctps = xmm_ctps[['srcname_fin', 'RA_fin', 'DEC_fin', 'pos_sigma_2d', 'flux_05-20', 'DET_LIKE_0', 'desi_id', 'desi_ra', 'desi_dec']]
csc_ctps = csc_ctps[['srcname_fin', 'RA_fin', 'DEC_fin', 'pos_sigma_2d', 'flux_05-20', 'DET_LIKE_0','desi_id', 'desi_ra', 'desi_dec']]
xmm_ctps['x_ray_det'] = 'xmm'
csc_ctps['x_ray_det'] = 'csc'
print('After Flux filter')
print('CSC companions', csc_ctps.shape[0])
print('XMM companions', xmm_ctps.shape[0])
print('Total companions possible', xmm_ctps.shape[0] + csc_ctps.shape[0])

After Flux filter
CSC companions 10
XMM companions 22
Total companions possible 32


In [38]:
final_ctps = pd.concat([xmm_ctps, csc_ctps])
final_ctps.sort_values(by='srcname_fin', inplace=True)
final_ctps.reset_index(drop=True, inplace=True)


final_ctps['n_csc_neigh'] = final_ctps['srcname_fin'].map(n_chandra_dict, na_action='ignore')
final_ctps['n_xmm_neigh'] = final_ctps['srcname_fin'].map(n_xmm_dict, na_action='ignore')
final_ctps['n_xmm_neigh'] = final_ctps['n_xmm_neigh'].fillna(0)
final_ctps['n_csc_neigh'] = final_ctps['n_csc_neigh'].fillna(0)


n_ctps = final_ctps.groupby('srcname_fin')['x_ray_det'].transform(lambda x: len(x.unique())) 
final_ctps['n_x_ray_det'] = n_ctps

n_desi_ctps = final_ctps.groupby('srcname_fin')['desi_id'].transform(lambda x: len(x.unique()))
final_ctps['n_desi_ctps'] = n_desi_ctps


final_ctps = final_ctps.query('( n_x_ray_det == 1 ) | ( n_x_ray_det==2 & n_desi_ctps == 1 )')
## final_ctps.query('( n_x_ray_det == 1 ) | ( n_x_ray_det==2 & n_desi_ctps == 1 )') - opposite of this


final_ctps.drop_duplicates(subset=['srcname_fin', 'desi_id'], inplace=True)


final_ctps = add_separation_columns(final_ctps, colname_ra1 = 'RA_fin', colname_dec1 = 'DEC_fin', colname_ra2 = 'desi_ra', colname_dec2 = 'desi_dec', colname = 'sep_ero_desi')



# #save final_ctps_csv to csv
final_ctps_csv = final_ctps[['srcname_fin', 'desi_id']]
final_ctps_csv.rename(columns={'desi_id':'desi_id_true'}, inplace=True)
final_ctps_csv.desi_id_true = 'hostless'
final_ctps_csv.to_csv(data_path+'validation_ctps_ero_desi_lh_hostless.csv', index=False)
print('Final number of hostless', final_ctps.shape[0])

#final_ctps

Final number of hostless 30


In [39]:
final_ctps = final_ctps.query(" ( (n_csc_neigh==0 & n_xmm_neigh==1) | (n_csc_neigh==1 & n_xmm_neigh==0) | (n_csc_neigh==1 & n_xmm_neigh==1) )")
print('After removing sources with more than one X-ray neighbor we have ', final_ctps.shape[0], ' sources')

After removing sources with more than one X-ray neighbor we have  30  sources


In [40]:
#combine with non-hostless list
df_ctps = pd.read_csv(data_path+'validation_ctps_ero_desi_lh_no_hostless.csv')
df_hostless = pd.read_csv(data_path+'validation_ctps_ero_desi_lh_hostless.csv')
df_ctps = pd.concat([df_ctps, df_hostless])

df_ctps.to_csv(data_path+'validation_ctps_ero_desi_lh.csv', index = False)
df_ctps.sample(10)

,srcname_fin,desi_id_true
338,SRGe J104837.1+585417,9011_614518_78
414,SRGe J105045.8+591943,9011_615262_1416
275,SRGe J104615.1+590736,9011_615260_277
376,SRGe J104928.0+582629,9011_613016_3798
187,SRGe J104218.4+583848,9011_613766_1997
367,SRGe J104919.0+584238,9011_613770_614
405,SRGe J105020.2+571422,9011_609169_889
323,SRGe J104803.0+591947,9011_615261_136
240,SRGe J104447.4+590632,9011_614516_176
216,SRGe J104355.4+593509,9011_615996_846
